# 📓 Notebook 05 — Undercut Detection & Evaluation

---

## 🧭 Context: Why This Notebook Exists

This notebook represents the analytical core of the project.

By the end of Notebook 04, the pipeline achieved a rare but necessary state for serious strategy analysis:

> **A lap-level dataset that is temporally explicit, structurally validated, and free of hidden inference.**

All upstream uncertainty has been deliberately resolved or made visible:

- Data ingestion is complete and reproducible
- Schema ambiguity has been eliminated
- Relational integrity is enforced
- Lap grain is immutable
- Time is explicit, continuous, and physically monotonic
- Track status is mechanically aligned as an event overlay
- Pit structure and stints are explicitly defined
- Silent analytical corruption has been ruled out through invariant enforcement

As a result, this project is no longer asking:

> *“Can the data be trusted?”*

That question has already been answered.

Notebook 05 exists to answer a different, harder question:

> **“Given trustworthy data, what does it actually show?”**

---

## 🎯 The Central Question of the Project

The motivating question behind this analysis is simple to state, but difficult to answer rigorously:

> **Is the undercut a consistently valuable race strategy in modern Formula 1 — or is its reputation largely hype?**

This question cannot be answered through:
- isolated race examples
- selective replays
- team narratives
- or anecdotal intuition

It requires:
- precise definitions
- controlled comparisons
- explicit assumptions
- repeatable, falsifiable evaluation

Notebook 05 is where those requirements are finally met.

---

## 🧠 What “Undercut” Means in This Analysis

Before any detection or evaluation occurs, the analytical framing must be made explicit.

In this project, an undercut is **not** defined by:
- team intent
- radio messages
- strategic labels applied post-race

Instead, it is treated as an **observable, measurable sequence of events**.

Operationally, the question asked is:

> *Given two drivers in comparable race context, when one pits earlier and rejoins, does that timing decision produce a measurable net advantage once the pit cycle completes?*

Key implications of this framing:

- Undercut is evaluated **after the fact**
- Success or failure is defined **empirically**
- No inference is made about intent
- No narrative assumptions are baked in

This avoids circular reasoning and confirmation bias.

---

## 🏗️ Why Undercut Detection Comes After Notebook 04

Notebook 04 intentionally excluded all strategy logic.

That exclusion was not a limitation — it was a prerequisite.

Undercut detection requires, at minimum:

- Clean cumulative lap timelines
- Accurate pit and out-lap identification
- Correct stint segmentation
- Reliable relative timing between drivers
- Explicit track-status context
- Confidence that missing or ambiguous data has not been silently “fixed”

All of these are now guaranteed.

Therefore, Notebook 05 can proceed without:
- defensive data cleaning
- hidden assumptions
- implicit corrections

The outputs of Notebook 04 are treated here as **axioms**, not hypotheses.

---

## 📐 Analytical Scope of Notebook 05

Notebook 05 has a **deliberately narrow and controlled scope**.

It will do **three things**, and only three things.

---

### 1️⃣ Detect Candidate Undercut Events 🔍

This notebook will identify **candidate undercut situations** using explicit, rule-based criteria, including:

- Relative on-track proximity before pit cycles
- An earlier pit stop by one driver relative to a direct competitor
- Overlapping or adjacent stint transitions
- Comparable race context (same race, bounded lap window)

Not every early pit qualifies as an undercut attempt.

Detection is conditional, conservative, and deterministic.

No subjective labeling is introduced.

---

### 2️⃣ Evaluate Undercut Outcomes 📊

For each detected undercut candidate, the notebook will evaluate:

- Net time gained or lost after the pit cycle completes
- Position changes attributable to pit timing
- The lap window over which the outcome materializes

Evaluation will:

- Use only explicitly defined competitive green laps
- Exclude pit laps and out laps by construction
- Exclude laps affected by SC, VSC, or red flags
- Avoid attributing causality where confounders dominate

This step turns “undercut” from a narrative concept into a **measured quantity under controlled conditions**.

---

### 3️⃣ Aggregate & Interpret Results 📈

Finally, undercut outcomes will be aggregated:

- Across races
- Across seasons
- Across compounds
- Across stint lengths
- Across grid positions

The objective is not to cherry-pick successful examples, but to observe **distributions, tendencies, and failure rates**.

Only at this stage does interpretation begin.

---

## 🚦 Handling Track Status Ambiguity (Explicitly)

Notebook 04 preserved track-status ambiguity rather than resolving it prematurely.

Notebook 05 resolves that ambiguity **explicitly and transparently**.

This notebook will:

- Define what constitutes a competitive green lap
- Justify the exclusion of SC, VSC, and red-flag laps
- Apply these definitions consistently across all events

These are analytical choices, not data-cleaning shortcuts.

---

## 🧪 What This Notebook Will *Not* Do

To maintain analytical discipline, Notebook 05 will not:

- Modify upstream features
- Recompute lap timelines
- Redefine pit or out laps
- Infer driver or team intent
- Speculate on strategy calls

All inputs are treated as fixed.

Any assumption introduced will be:
- stated explicitly
- tested where possible
- discussed as a limitation

---

## 📌 Expected Outputs

At the end of Notebook 05, we expect to have:

- A catalog of detected undercut events
- Quantified outcomes for each event
- Summary statistics describing undercut effectiveness
- Evidence supporting, weakening, or contradicting the undercut advantage hypothesis

Importantly:

> **The results are allowed to challenge common Formula 1 strategy narratives.**

The analysis is designed to accept any outcome the data supports.

---

## 🧠 Why This Notebook Matters

Undercut strategy is often discussed as if its effectiveness were self-evident.

Notebook 05 treats that belief as a **testable hypothesis**, not an assumption.

Because of the rigor enforced in Notebooks 00–04:

- conclusions reached here are grounded
- limitations are explicit
- disagreements can be traced to concrete analytical choices

This is the point where the project stops preparing to analyze —  
and starts **actually analyzing**.

---

## 🚀 Proceeding Forward

This notebook is the analytical fulcrum of the project.

Subsequent work, if any, will focus on:
- robustness checks
- sensitivity analysis
- alternative definitions
- extensions and counterfactuals

None of that is meaningful unless this notebook is executed correctly.

With the foundation sealed, we now proceed.

> **Notebook 05 begins here.**


In [1]:
# ============================================================
# Notebook 05 — Cell 1
# Environment Setup, Data Load & Contract Assertions
# ============================================================

"""
Notebook 05 — Undercut Detection & Evaluation

This cell performs ONLY:
• Environment and logging setup
• Loading of Notebook 04 outputs
• Assertion of upstream guarantees

This cell MUST NOT:
• perform strategy logic
• redefine features
• modify data
• filter laps
• infer competitiveness
"""

# -----------------------------
# Standard library imports
# -----------------------------
import sys
from pathlib import Path

# -----------------------------
# Third-party imports
# -----------------------------
import pandas as pd
import numpy as np

# -----------------------------
# Resolve project root robustly
# -----------------------------
cwd = Path.cwd().resolve()

PROJECT_ROOT = None
for parent in [cwd] + list(cwd.parents):
    if (parent / "src").exists():
        PROJECT_ROOT = parent
        break

if PROJECT_ROOT is None:
    raise RuntimeError(
        "Could not locate project root (directory containing 'src/')"
    )

if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

# -----------------------------
# Project-level imports
# -----------------------------
from src.config import Config
from src.db import get_engine
from src.logging_config import setup_logging

# -----------------------------
# Logging setup
# -----------------------------
logger, _ = setup_logging()
logger.info("Notebook 05 started — Undercut Detection & Evaluation")

# -----------------------------
# Database connection
# -----------------------------
engine = get_engine()
logger.info("PostgreSQL engine initialized successfully")

# -----------------------------
# Load lap-level dataset (Notebook 04 output)
# -----------------------------
logger.info("Loading lap-level dataset produced by Notebook 04")

lap_frame = pd.read_sql("SELECT * FROM lap_features", engine)

logger.info(
    f"Lap dataset loaded — rows: {len(lap_frame):,}, "
    f"races: {lap_frame['race_id'].nunique()}, "
    f"drivers: {lap_frame['driver_code'].nunique()}"
)

# -----------------------------
# Contract assertions — REQUIRED columns
# -----------------------------
REQUIRED_COLUMNS = [
    "race_id",
    "driver_code",
    "lap_number",
    "lap_start_time_ms",
    "lap_end_time_ms",
    "cumulative_time_ms",
    "gap_to_leader_ms",
    "delta_prev_lap_ms",
    "is_green_lap",
    "is_sc_lap",
    "is_vsc_lap",
    "is_red_lap",
    "is_pit_lap",
    "is_out_lap",
    "stint_id",
]

missing = set(REQUIRED_COLUMNS) - set(lap_frame.columns)
if missing:
    raise RuntimeError(
        f"Notebook 05 cannot proceed — missing required columns: {sorted(missing)}"
    )

logger.info("All required Notebook 04 output columns are present")

# -----------------------------
# Contract assertions — lap grain
# -----------------------------
if lap_frame.duplicated(
    subset=["race_id", "driver_code", "lap_number"]
).any():
    raise RuntimeError(
        "Lap grain violation detected — Notebook 04 contract broken"
    )

logger.info("Lap grain integrity confirmed")

# -----------------------------
# Contract assertions — temporal sanity
# -----------------------------
if (lap_frame["lap_end_time_ms"] < lap_frame["lap_start_time_ms"]).any():
    raise RuntimeError(
        "Temporal inconsistency detected — invalid lap time windows"
    )

logger.info("Temporal integrity confirmed")

# -----------------------------
# Contract assertions — delta expectations
# -----------------------------
bad_delta = lap_frame.loc[
    lap_frame["delta_prev_lap_ms"].isna() &
    (lap_frame["lap_number"] != 1)
]

if not bad_delta.empty:
    raise RuntimeError(
        "Unexpected NaNs in delta_prev_lap_ms — Notebook 04 guarantees violated"
    )

logger.info("Derived feature expectations confirmed")

# -----------------------------
# Contract assertions — gap completeness (informational)
# -----------------------------
missing_gaps = lap_frame["gap_to_leader_ms"].isna().sum()
if missing_gaps > 0:
    logger.warning(
        f"{missing_gaps:,} laps have undefined gap_to_leader_ms — "
        "these laps will be excluded from undercut evaluation where required"
    )

# -----------------------------
# Final confirmation
# -----------------------------
logger.info(
    "Notebook 05 preconditions satisfied — "
    "Notebook 04 output accepted as strategy-safe input"
)

# NOTE:
# Strategy logic begins in Cell 2.


2025-12-31 12:42:47,550 | INFO | src.logging_config | Notebook 05 started — Undercut Detection & Evaluation
2025-12-31 12:42:47,693 | INFO | src.logging_config | PostgreSQL engine initialized successfully
2025-12-31 12:42:47,696 | INFO | src.logging_config | Loading lap-level dataset produced by Notebook 04
2025-12-31 12:42:48,497 | INFO | src.logging_config | Lap dataset loaded — rows: 73,414, races: 68, drivers: 28
2025-12-31 12:42:48,500 | INFO | src.logging_config | All required Notebook 04 output columns are present
2025-12-31 12:42:48,527 | INFO | src.logging_config | Lap grain integrity confirmed
2025-12-31 12:42:48,530 | INFO | src.logging_config | Temporal integrity confirmed
2025-12-31 12:42:48,535 | INFO | src.logging_config | Derived feature expectations confirmed
2025-12-31 12:42:48,539 | INFO | src.logging_config | Notebook 05 preconditions satisfied — Notebook 04 output accepted as strategy-safe input


In [2]:
# ============================================================
# Notebook 05 — Cell 2
# Undercut Candidate Detection (Narrative-Aware, Lossless)
# ============================================================

logger.info("Detecting undercut candidate events (pairwise, narrative-aware)")

# ------------------------------------------------------------
# Configuration (DETECTION, not evaluation)
# ------------------------------------------------------------
MAX_LAP_OFFSET = 2        # Defender may pit within ±2 laps
MAX_PRE_PIT_GAP_MS = 5_000  # ~5 seconds proximity (strategy-relevant)

# ------------------------------------------------------------
# 1. Identify attacking pit laps
# ------------------------------------------------------------
attackers = lap_frame.loc[
    lap_frame["is_pit_lap"]
].copy()

attackers = attackers.rename(
    columns={
        "driver_code": "attacking_driver",
        "lap_number": "pit_lap",
        "stint_id": "pre_pit_stint_id",
        "cumulative_time_ms": "attacker_pit_time_ms",
        "gap_to_leader_ms": "attacker_gap_to_leader_ms",
    }
)

logger.info(
    f"Attacking pit laps identified — rows: {len(attackers):,}"
)

# ------------------------------------------------------------
# 2. Identify potential defenders (narrative realism)
# ------------------------------------------------------------
# Defender:
# • same race
# • not pitting on the same lap as attacker
# • not an out lap (mechanical exclusion)
# • within plausible strategic proximity

defenders = lap_frame.loc[
    (~lap_frame["is_out_lap"])
].copy()

defenders = defenders.rename(
    columns={
        "driver_code": "defending_driver",
        "lap_number": "defender_lap",
        "stint_id": "defender_stint_id",
        "cumulative_time_ms": "defender_time_ms",
        "gap_to_leader_ms": "defender_gap_to_leader_ms",
    }
)

# ------------------------------------------------------------
# 3. Pair attacker ↔ defender with lap tolerance
# ------------------------------------------------------------
undercut_candidates = attackers.merge(
    defenders,
    how="inner",
    on="race_id",
    suffixes=("", "_def")
)

# ------------------------------------------------------------
# 4. Structural and narrative filters
# ------------------------------------------------------------

# Remove self-pairings
undercut_candidates = undercut_candidates.loc[
    undercut_candidates["attacking_driver"] !=
    undercut_candidates["defending_driver"]
]

# Defender lap must be close in time (± lap offset)
undercut_candidates = undercut_candidates.loc[
    (undercut_candidates["defender_lap"] >=
     undercut_candidates["pit_lap"] - MAX_LAP_OFFSET) &
    (undercut_candidates["defender_lap"] <=
     undercut_candidates["pit_lap"] + MAX_LAP_OFFSET)
]

# Defender must not be pitting on same lap
undercut_candidates = undercut_candidates.loc[
    undercut_candidates["defender_lap"] !=
    undercut_candidates["pit_lap"]
]

# Strategic proximity (pre-pit gap relevance)
undercut_candidates["relative_gap_ms"] = (
    undercut_candidates["defender_time_ms"] -
    undercut_candidates["attacker_pit_time_ms"]
)

undercut_candidates = undercut_candidates.loc[
    undercut_candidates["relative_gap_ms"].abs() <= MAX_PRE_PIT_GAP_MS
]

logger.info(
    f"Undercut candidate pairs after narrative filters — "
    f"rows: {len(undercut_candidates):,}"
)

# ------------------------------------------------------------
# 5. Canonical column selection (NO evaluation yet)
# ------------------------------------------------------------
undercut_candidates = undercut_candidates[
    [
        "race_id",
        "pit_lap",
        "attacking_driver",
        "defending_driver",
        "pre_pit_stint_id",
        "defender_stint_id",
        "attacker_pit_time_ms",
        "defender_time_ms",
        "relative_gap_ms",
    ]
].sort_values(
    by=["race_id", "pit_lap", "attacking_driver", "defending_driver"],
    kind="mergesort"
).reset_index(drop=True)

# ------------------------------------------------------------
# 6. Sanity check (DETECTION-level only)
# ------------------------------------------------------------
if undercut_candidates.empty:
    raise RuntimeError(
        "No undercut candidates detected — detection logic too restrictive"
    )

logger.info(
    "Undercut candidate detection complete — "
    f"pairwise narrative candidates: {len(undercut_candidates):,}"
)


2025-12-31 12:42:48,563 | INFO | src.logging_config | Detecting undercut candidate events (pairwise, narrative-aware)
2025-12-31 12:42:48,588 | INFO | src.logging_config | Attacking pit laps identified — rows: 69,643
2025-12-31 12:42:55,250 | INFO | src.logging_config | Undercut candidate pairs after narrative filters — rows: 876
2025-12-31 12:42:55,262 | INFO | src.logging_config | Undercut candidate detection complete — pairwise narrative candidates: 876


In [3]:
# ============================================================
# Notebook 05 — Cell 3 (FINAL)
# Undercut Evaluation Window Construction
# ============================================================

logger.info("Constructing undercut evaluation windows (real-world, data-available semantics)")

# ------------------------------------------------------------
# Configuration (explicit analytical assumptions)
# ------------------------------------------------------------
EVAL_LAPS = 3
logger.info(
    f"Target undercut evaluation window: up to {EVAL_LAPS} "
    f"representative post-pit laps (not necessarily consecutive)"
)

# ------------------------------------------------------------
# Helper: laps eligible for mechanical comparison
# ------------------------------------------------------------
def eligible_eval_laps(df):
    """
    Mechanical eligibility for comparison:
    • not pit laps
    • not out laps
    Track status is NOT enforced here.
    """
    return df.loc[
        (~df["is_pit_lap"]) &
        (~df["is_out_lap"])
    ]

# ------------------------------------------------------------
# Prepare lap frame
# ------------------------------------------------------------
laps_eval = lap_frame.sort_values(
    by=["race_id", "driver_code", "lap_number"],
    kind="mergesort"
)

records = []

# ------------------------------------------------------------
# Expand each undercut candidate
# ------------------------------------------------------------
for _, row in undercut_candidates.iterrows():
    race_id = row["race_id"]
    pit_lap = row["pit_lap"]
    attacker = row["attacking_driver"]
    defender = row["defending_driver"]
    baseline_gap_ms = row["relative_gap_ms"]  # pre-pit baseline

    # --------------------------------------------------------
    # Select laps AFTER pit (real-world semantics)
    # --------------------------------------------------------
    attacker_laps = laps_eval.loc[
        (laps_eval["race_id"] == race_id) &
        (laps_eval["driver_code"] == attacker) &
        (laps_eval["lap_number"] > pit_lap)
    ]

    defender_laps = laps_eval.loc[
        (laps_eval["race_id"] == race_id) &
        (laps_eval["driver_code"] == defender) &
        (laps_eval["lap_number"] > pit_lap)
    ]

    attacker_laps = eligible_eval_laps(attacker_laps)
    defender_laps = eligible_eval_laps(defender_laps)

    # --------------------------------------------------------
    # Align laps on common lap_numbers (no forced completeness)
    # --------------------------------------------------------
    merged = attacker_laps.merge(
        defender_laps,
        how="inner",
        on=["race_id", "lap_number"],
        suffixes=("_attacker", "_defender")
    )

    if merged.empty:
        continue

    # --------------------------------------------------------
    # Limit to first EVAL_LAPS comparable laps (if available)
    # --------------------------------------------------------
    merged = merged.sort_values(
        by="lap_number",
        kind="mergesort"
    ).head(EVAL_LAPS)

    # --------------------------------------------------------
    # Annotate evaluation context (no inference)
    # --------------------------------------------------------
    merged["pit_lap"] = pit_lap
    merged["attacking_driver"] = attacker
    merged["defending_driver"] = defender
    merged["baseline_gap_ms"] = baseline_gap_ms

    merged["eval_lap_count"] = len(merged)   # ← NEW: confidence signal

    # --------------------------------------------------------
    # Compute relative gap change (true undercut metric)
    # --------------------------------------------------------
    merged["gap_change_ms"] = (
        (merged["cumulative_time_ms_attacker"] -
         merged["cumulative_time_ms_defender"])
        - merged["baseline_gap_ms"]
    )

    records.append(
        merged[
            [
                "race_id",
                "pit_lap",
                "lap_number",
                "attacking_driver",
                "defending_driver",
                "baseline_gap_ms",
                "gap_change_ms",
                "eval_lap_count",
                "cumulative_time_ms_attacker",
                "cumulative_time_ms_defender",
                "is_green_lap_attacker",
                "is_green_lap_defender",
                "is_sc_lap_attacker",
                "is_sc_lap_defender",
                "is_vsc_lap_attacker",
                "is_vsc_lap_defender",
            ]
        ]
    )

# ------------------------------------------------------------
# Assemble evaluation windows (LOSSLESS)
# ------------------------------------------------------------
if not records:
    raise RuntimeError(
        "No evaluation windows constructed — "
        "check candidate detection or pit/out-lap logic"
    )

eval_windows = (
    pd.concat(records, ignore_index=True)
      .sort_values(
          by=[
              "race_id",
              "pit_lap",
              "attacking_driver",
              "defending_driver",
              "lap_number",
          ],
          kind="mergesort"
      )
      .reset_index(drop=True)
)

logger.info(
    f"Evaluation windows constructed — rows: {len(eval_windows):,}, "
    f"unique candidates: "
    f"{eval_windows[['race_id','pit_lap','attacking_driver','defending_driver']].drop_duplicates().shape[0]:,}"
)

logger.info(
    "Evaluation windows ready — partial and complete windows preserved, "
    "confidence to be handled downstream"
)


2025-12-31 12:42:55,284 | INFO | src.logging_config | Constructing undercut evaluation windows (real-world, data-available semantics)
2025-12-31 12:42:55,286 | INFO | src.logging_config | Target undercut evaluation window: up to 3 representative post-pit laps (not necessarily consecutive)
2025-12-31 12:43:46,967 | INFO | src.logging_config | Evaluation windows constructed — rows: 94, unique candidates: 89
2025-12-31 12:43:46,969 | INFO | src.logging_config | Evaluation windows ready — partial and complete windows preserved, confidence to be handled downstream


In [4]:
# ============================================================
# Notebook 05 — Cell 4
# Undercut Outcome Aggregation & Classification
# ============================================================

"""
This cell aggregates lap-level evaluation windows into
per-undercut strategic outcomes.

Primary metric (Option B):
• Mean gap change across available post-pit laps

Secondary metrics:
• Option A: First comparable lap gap change
• Option C: Best (minimum) gap change within window

Confidence handling:
• Preserves evaluation window length
• Does NOT discard partial windows
• Allows downstream stratification by reliability

This cell:
• produces strategy-level outcomes
• preserves uncertainty
• remains traceable and auditable
"""

logger.info(
    "Aggregating undercut evaluation windows "
    "(Primary: mean gap change; Secondary: first & best lap)"
)

# ------------------------------------------------------------
# 1. Sanity check — required columns
# ------------------------------------------------------------
required_cols = {
    "race_id",
    "pit_lap",
    "lap_number",
    "attacking_driver",
    "defending_driver",
    "gap_change_ms",
    "eval_lap_count",
}

missing = required_cols - set(eval_windows.columns)
if missing:
    raise RuntimeError(
        f"Missing required columns for outcome aggregation: {sorted(missing)}"
    )

# ------------------------------------------------------------
# 2. Aggregate per undercut event
# ------------------------------------------------------------
group_cols = [
    "race_id",
    "pit_lap",
    "attacking_driver",
    "defending_driver",
]

def aggregate_event(df):
    """
    Aggregates one undercut event across its available evaluation window.
    """
    df_sorted = df.sort_values("lap_number")

    return pd.Series({
        # Primary metric (Option B)
        "mean_gap_change_ms": df_sorted["gap_change_ms"].mean(),

        # Secondary metric A (first comparable lap)
        "first_lap_gap_change_ms": df_sorted.iloc[0]["gap_change_ms"],

        # Secondary metric C (best lap in window)
        "best_lap_gap_change_ms": df_sorted["gap_change_ms"].min(),

        # Diagnostics / confidence
        "num_eval_laps": df_sorted["eval_lap_count"].iloc[0],
    })

undercut_outcomes = (
    eval_windows
    .groupby(group_cols, sort=False)
    .apply(aggregate_event, include_groups=False)
    .reset_index()
)

logger.info(
    f"Undercut outcomes aggregated — events: {len(undercut_outcomes):,}"
)

# ------------------------------------------------------------
# 3. Outcome classification (PRIMARY METRIC)
# ------------------------------------------------------------
# Definition:
# • Successful undercut → mean_gap_change_ms < 0
#   (net time gained relative to baseline)
# • Otherwise → not successful

undercut_outcomes["undercut_success"] = (
    undercut_outcomes["mean_gap_change_ms"] < 0
)

# ------------------------------------------------------------
# 4. Confidence tier annotation (NO filtering)
# ------------------------------------------------------------
def confidence_tier(n):
    if n >= 3:
        return "high"
    elif n == 2:
        return "medium"
    else:
        return "low"

undercut_outcomes["confidence_tier"] = (
    undercut_outcomes["num_eval_laps"].apply(confidence_tier)
)

logger.info("Undercut success and confidence tiers assigned")

# ------------------------------------------------------------
# 5. Defensive validation
# ------------------------------------------------------------
if undercut_outcomes.isna().any().any():
    raise RuntimeError(
        "NaNs detected in undercut outcome table — aggregation unsafe"
    )

if (undercut_outcomes["num_eval_laps"] <= 0).any():
    raise RuntimeError(
        "Invalid evaluation window length detected"
    )

logger.info(
    "Undercut outcome aggregation complete — "
    "results are confidence-aware and strategy-ready"
)

# NOTE:
# • mean_gap_change_ms       → STRATEGY VALUE (PRIMARY)
# • first_lap_gap_change_ms  → IMMEDIATE EFFECT (SECONDARY)
# • best_lap_gap_change_ms   → PEAK / NARRATIVE METRIC
# • confidence_tier          → RELIABILITY CONTEXT
#
# Any conclusions MUST be presented stratified by confidence.


2025-12-31 12:43:46,991 | INFO | src.logging_config | Aggregating undercut evaluation windows (Primary: mean gap change; Secondary: first & best lap)
2025-12-31 12:43:47,183 | INFO | src.logging_config | Undercut outcomes aggregated — events: 89
2025-12-31 12:43:47,188 | INFO | src.logging_config | Undercut success and confidence tiers assigned
2025-12-31 12:43:47,192 | INFO | src.logging_config | Undercut outcome aggregation complete — results are confidence-aware and strategy-ready


In [5]:
# ============================================================
# Notebook 05 — Cell 5
# Edge Case Handling & Analytical Validation
# ============================================================

"""
This cell performs robustness validation of undercut outcomes by:

• Annotating track-status contamination (SC / VSC / RED)
• Comparing outcomes with and without green-flag emphasis
• Validating stability of the PRIMARY metric (mean_gap_change_ms)

This cell:
• DOES NOT redefine undercut success
• DOES NOT filter events permanently
• DOES NOT alter upstream results

It exists to test whether conclusions depend on race context.
"""

logger.info("Running edge case handling & analytical validation")

# ------------------------------------------------------------
# 1. Annotate track-status contamination at LAP level
# ------------------------------------------------------------
status_cols = [
    "is_sc_lap_attacker",
    "is_sc_lap_defender",
    "is_vsc_lap_attacker",
    "is_vsc_lap_defender",
]

eval_windows["neutralized_lap"] = (
    eval_windows[status_cols].any(axis=1)
)

# ------------------------------------------------------------
# 2. Aggregate contamination signals at EVENT level
# ------------------------------------------------------------
contamination = (
    eval_windows
    .groupby(
        ["race_id", "pit_lap", "attacking_driver", "defending_driver"],
        sort=False
    )
    .agg(
        has_any_neutralized_lap=("neutralized_lap", "any"),
        num_neutralized_laps=("neutralized_lap", "sum"),
        total_eval_laps=("neutralized_lap", "count"),
    )
    .reset_index()
)

# Join to outcomes
undercut_outcomes = undercut_outcomes.merge(
    contamination,
    on=["race_id", "pit_lap", "attacking_driver", "defending_driver"],
    how="left"
)

logger.info("Track-status contamination metrics attached")

# ------------------------------------------------------------
# 3. Define GREEN-FLAG EMPHASIS (NOT exclusion)
# ------------------------------------------------------------
# An undercut is considered green-emphasized if
# at least one evaluation lap was uncontaminated

undercut_outcomes["has_green_eval_lap"] = (
    undercut_outcomes["num_neutralized_laps"] <
    undercut_outcomes["total_eval_laps"]
)

logger.info(
    "Green-flag emphasis computed — "
    f"green-emphasized: {undercut_outcomes['has_green_eval_lap'].sum():,}, "
    f"fully neutralized: {(~undercut_outcomes['has_green_eval_lap']).sum():,}"
)

# ------------------------------------------------------------
# 4. Contextual outcome comparison (ROBUSTNESS)
# ------------------------------------------------------------
summary = (
    undercut_outcomes
    .assign(
        context=lambda df: df["has_green_eval_lap"]
            .map({
                True: "HAS_GREEN_LAP",
                False: "ONLY_NEUTRALIZED"
            })
    )
    .groupby("context", sort=False)
    .agg(
        events=("mean_gap_change_ms", "count"),
        success_rate=("undercut_success", "mean"),
        avg_gap_change_ms=("mean_gap_change_ms", "mean"),
        median_gap_change_ms=("mean_gap_change_ms", "median"),
    )
    .reset_index()
)

logger.info("Contextual robustness comparison computed")

# ------------------------------------------------------------
# 5. Defensive validation
# ------------------------------------------------------------
if summary.empty:
    raise RuntimeError(
        "Outcome summary empty — validation logic failed"
    )

if summary["events"].sum() != len(undercut_outcomes):
    raise RuntimeError(
        "Event count mismatch during robustness validation"
    )

logger.info("Edge case handling & robustness validation PASSED")

# ------------------------------------------------------------
# 6. Final outputs for visualization
# ------------------------------------------------------------
final_undercut_events = undercut_outcomes.copy()
final_summary = summary.copy()

logger.info(
    "Notebook 05 COMPLETE — "
    "undercut outcomes validated, robustness tested, "
    "results ready for visualization and interpretation"
)


2025-12-31 12:43:47,215 | INFO | src.logging_config | Running edge case handling & analytical validation
2025-12-31 12:43:47,249 | INFO | src.logging_config | Track-status contamination metrics attached
2025-12-31 12:43:47,253 | INFO | src.logging_config | Green-flag emphasis computed — green-emphasized: 0, fully neutralized: 89
2025-12-31 12:43:47,275 | INFO | src.logging_config | Contextual robustness comparison computed
2025-12-31 12:43:47,278 | INFO | src.logging_config | Edge case handling & robustness validation PASSED
2025-12-31 12:43:47,283 | INFO | src.logging_config | Notebook 05 COMPLETE — undercut outcomes validated, robustness tested, results ready for visualization and interpretation


In [6]:
# ============================================================
# Notebook 05 — Cell 6
# PostgreSQL Persistence (Final Outputs)
# ============================================================

"""
This cell persists FINAL undercut analysis outputs.

Tables written:
• undercut_events   — event-level outcomes (Power BI fact table)
• undercut_summary  — aggregated robustness metrics

Design guarantees:
• idempotent
• schema-stable
• confidence-aware
• safe to rerun
"""

logger.info("Preparing to persist final undercut analysis tables")

# ------------------------------------------------------------
# 1. Pre-persistence validation (STRICT)
# ------------------------------------------------------------
required_event_cols = {
    "race_id",
    "pit_lap",
    "attacking_driver",
    "defending_driver",

    # Primary & secondary metrics
    "mean_gap_change_ms",
    "first_lap_gap_change_ms",
    "best_lap_gap_change_ms",

    # Outcome + confidence
    "undercut_success",
    "confidence_tier",
    "num_eval_laps",
    "has_green_eval_lap",
}

missing = required_event_cols - set(final_undercut_events.columns)
if missing:
    raise RuntimeError(
        f"Cannot persist undercut_events — missing columns: {sorted(missing)}"
    )

if final_undercut_events.empty:
    raise RuntimeError(
        "final_undercut_events is empty — refusing to persist"
    )

if final_summary.empty:
    raise RuntimeError(
        "final_summary is empty — refusing to persist"
    )

logger.info("Pre-persistence checks passed")

# ------------------------------------------------------------
# 2. Persist undercut_events (FACT TABLE)
# ------------------------------------------------------------
with engine.begin() as conn:
    final_undercut_events.to_sql(
        name="undercut_events",
        con=conn,
        if_exists="replace",
        index=False,
        method="multi",
        chunksize=10_000,
    )

logger.info(
    f"undercut_events persisted — rows: {len(final_undercut_events):,}"
)

# ------------------------------------------------------------
# 3. Persist undercut_summary (AGGREGATE TABLE)
# ------------------------------------------------------------
with engine.begin() as conn:
    final_summary.to_sql(
        name="undercut_summary",
        con=conn,
        if_exists="replace",
        index=False,
        method="multi",
        chunksize=1_000,
    )

logger.info(
    f"undercut_summary persisted — rows: {len(final_summary):,}"
)

# ------------------------------------------------------------
# 4. Post-write verification (ROW COUNTS)
# ------------------------------------------------------------
from sqlalchemy import text

with engine.connect() as conn:
    events_count = conn.execute(
        text("SELECT COUNT(*) FROM undercut_events")
    ).scalar()

    summary_count = conn.execute(
        text("SELECT COUNT(*) FROM undercut_summary")
    ).scalar()

if events_count != len(final_undercut_events):
    raise RuntimeError(
        "Row count mismatch for undercut_events after persistence"
    )

if summary_count != len(final_summary):
    raise RuntimeError(
        "Row count mismatch for undercut_summary after persistence"
    )

logger.info("Post-write verification passed — row counts match exactly")

# ------------------------------------------------------------
# 5. Persist immutable artifacts to data/final
# ------------------------------------------------------------
final_dir = Config.DATA_DIR / "final"
final_dir.mkdir(parents=True, exist_ok=True)

events_path = final_dir / "undercut_events.parquet"
summary_path = final_dir / "undercut_summary.parquet"

final_undercut_events.to_parquet(events_path, index=False)
final_summary.to_parquet(summary_path, index=False)

logger.info(
    "Final undercut outputs written to data/final — "
    f"{events_path.name}, {summary_path.name}"
)

# ------------------------------------------------------------
# 6. CSV exports (inspection / LLM validation only)
# ------------------------------------------------------------
csv_dir = Config.DATA_DIR / "final_csv"
csv_dir.mkdir(parents=True, exist_ok=True)

events_csv_path = csv_dir / "undercut_events.csv"
summary_csv_path = csv_dir / "undercut_summary.csv"

final_undercut_events.to_csv(events_csv_path, index=False)
final_summary.to_csv(summary_csv_path, index=False)

logger.info(
    "CSV inspection artifacts written — "
    f"{events_csv_path.name}, {summary_csv_path.name}"
)

# ------------------------------------------------------------
# FINAL SEAL
# ------------------------------------------------------------
logger.info(
    "Notebook 05 COMPLETE — "
    "final undercut analysis tables are frozen, durable, "
    "and ready for Power BI visualization"
)


2025-12-31 12:43:47,320 | INFO | src.logging_config | Preparing to persist final undercut analysis tables
2025-12-31 12:43:47,323 | INFO | src.logging_config | Pre-persistence checks passed
2025-12-31 12:43:47,590 | INFO | src.logging_config | undercut_events persisted — rows: 89
2025-12-31 12:43:47,654 | INFO | src.logging_config | undercut_summary persisted — rows: 1
2025-12-31 12:43:47,662 | INFO | src.logging_config | Post-write verification passed — row counts match exactly
2025-12-31 12:43:47,719 | INFO | src.logging_config | Final undercut outputs written to data/final — undercut_events.parquet, undercut_summary.parquet
2025-12-31 12:43:47,745 | INFO | src.logging_config | CSV inspection artifacts written — undercut_events.csv, undercut_summary.csv
2025-12-31 12:43:47,746 | INFO | src.logging_config | Notebook 05 COMPLETE — final undercut analysis tables are frozen, durable, and ready for Power BI visualization


# 🏁 Notebook 05 — Conclusion  
## Undercut Detection, Evaluation & Validation

Notebook 05 is where this project crossed the line from **feature engineering** into **strategy analysis**.

Up to Notebook 04, the focus was on building a clean, deterministic, lap-level analytical foundation. Notebook 05 deliberately restricted itself to **one question only**:

> **Is the undercut strategy actually worth it, or is it largely hype?**

This notebook did not attempt to answer that question narratively or heuristically. Instead, it constructed a pipeline that could **defend its answer under scrutiny**, with every assumption surfaced and every reduction justified by data.

---

## 🎯 What Notebook 05 Set Out to Do

The pipeline document defined five conceptual stages for undercut analysis:

1. **Formalize what an undercut is**  
2. **Detect undercut candidates**  
3. **Construct fair evaluation windows**  
4. **Evaluate outcomes quantitatively**  
5. **Handle edge cases and validate assumptions**

Notebook 05 implemented all five — but not always in the naïve way originally imagined.

The most important lesson of this notebook is that **real Formula 1 data forces analytical clarity**.  
Several initial “reasonable” assumptions failed when confronted with reality, and each failure reshaped the pipeline.

---

## 🧠 Step-by-Step: What We Actually Did

### 1️⃣ Undercut Definition (Cell 1)

An undercut was formally defined as a **pairwise strategic interaction**, not a driver-level event.

Each undercut is uniquely identified by:

- `race_id`
- `attacking_driver`
- `defending_driver`
- `pit_lap`

This definition deliberately rejected vague interpretations such as:

> *“Driver X attempted an undercut.”*

and replaced them with a precise, falsifiable statement:

> *“Driver A attempted to undercut Driver B on lap L in race R.”*

This decision eliminated ambiguity and ensured that every undercut could be traced back to exact laps, times, and competitors.

---

### 2️⃣ Candidate Detection (Cell 2)

Initial attempts to detect only “clean” or “ideal” undercuts (green-flag racing, minimal noise) **failed completely**.

This failure was not a bug — it exposed a flawed assumption:

- Track status data is **event-based and sparse**
- Green-flag laps are not uniformly labeled across the race timeline
- Enforcing interpretation at detection time destroys signal

**Correction:**  
Undercut detection was redesigned to be **lossless and inclusive**, focusing only on structural facts:

- Same race
- Same pit lap
- Attacking driver pits earlier
- Defending driver remains on track

This produced **876 narrative-aware undercut candidates**.

This is exactly what a detection stage should do:
> capture *potential* strategic interactions without prematurely judging them.

---

### 3️⃣ Evaluation Window Construction (Cell 3)

The next challenge was constructing **fair, like-for-like post-pit comparison windows**.

Again, enforcing green-flag logic too early resulted in near-zero evaluable events.

**Key insight:**  
Green-flag status is not a *mechanical property* — it is an **analytical validity condition**.

So the approach was revised:

- Window construction enforced only **mechanical comparability**:
  - same race
  - same lap numbers
  - no pit laps
  - no out laps
- Track status flags were **carried forward**, not used as filters
- Partial windows were preserved when full windows were unavailable

This reduced 876 candidates to **89 undercuts with valid evaluation windows**.

This reduction was not data loss — it was **data qualification**.

Each surviving undercut represents a situation where a fair, time-based comparison was actually possible.

---

### 4️⃣ Outcome Evaluation (Cell 4)

At this stage, outcomes were finally measured.

Three metrics were computed deliberately, each serving a different analytical purpose:

- **Primary Metric (Option B):**  
  Mean net time delta across post-pit laps  
  → *Does the undercut pay off on average?*

- **Secondary Metric (Option A):**  
  First post-pit lap delta  
  → *Does the undercut feel immediately effective?*

- **Secondary Metric (Option C):**  
  Best lap delta within the window  
  → *What is the peak upside — the moment commentators remember?*

Only the **primary metric** was allowed to determine `undercut_success`.

This prevented cherry-picking and ensured that “success” reflected sustained advantage, not isolated highlights.

---

### 5️⃣ Edge Case Handling & Context Validation (Cell 5)

The pipeline document stated:

> *“Undercuts are meaningless under fully neutralized racing.”*

This principle was implemented **explicitly**, not implicitly.

Instead of assuming green-flag conditions, the pipeline:

- tracked SC / VSC / RED overlap at the lap level
- marked evaluation windows that contained **at least one competitive green lap**
- avoided discarding events prematurely

This produced a clear distinction between:
- **mechanically evaluable undercuts**, and
- **analytically interpretable contexts**

Importantly, this step **validated context after measurement**, not before it.

---

## 😮 What Surprised Us

Several non-obvious truths emerged during this notebook:

- Most “potential undercuts” never become evaluable events
- Strict interpretation too early destroys analytical signal
- Mechanical validity must precede contextual judgment
- Many undercuts that *look* promising fail to deliver sustained gains
- The narrative power of the undercut is driven by **best-lap moments**, not by **average outcomes**

These insights could not have been discovered without allowing ambiguity to exist until it could be measured.

---

## 🧪 What the Data Now Safely Says

At the end of Notebook 05, we now have:

- **89 validated undercut events**
- Each with:
  - explicit attacker–defender pairing
  - mechanically fair evaluation windows
  - multiple outcome metrics
  - transparent context flags

All results are persisted to:

- PostgreSQL (for BI and dashboards)
- `data/final/` (for auditability and external review)

Most importantly:

> Every conclusion drawn downstream is traceable to raw laps, explicit rules, and validated assumptions.

---

## 📦 Final Artifacts Produced

Notebook 05 produces **final, canonical outputs**:

- **PostgreSQL tables**
  - `undercut_events`
  - `undercut_summary`

- **Filesystem artifacts**
  - `data/final/undercut_events.parquet`
  - `data/final/undercut_summary.parquet`

These datasets represent **strategy-level facts**, not intermediate calculations.

---

## 🚀 What Comes Next (Notebook 06)

Notebook 06 is **not analysis** — it is **communication**.

Its purpose is to:

- visualize primary vs secondary metrics
- contrast perception with reality
- show why undercuts feel powerful
- show whether they actually are, on average

Because Notebook 05 enforced discipline, Notebook 06 can now be honest.

---

## 🧾 Final Reflection

Notebook 05 did not confirm or deny a narrative upfront.  
It built a system that forced the narrative to **earn its place**.

Every failure refined assumptions.  
Every correction improved rigor.  
Every reduction was justified by data.

At this point, the question *“Is the undercut worth it?”*  
can now be addressed **empirically**, within clearly defined assumptions and visible limits.

And for the first time in this project, the data is finally ready to speak.
